<a href="https://colab.research.google.com/github/Fredooooooo/JSC270_Assg4/blob/main/270A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
api_key = 'BsEu3jVTUIwLfVnav0qZsS12j'

api_secret_key = 'mgeSmV9zffc1CXYR4geaFQx3PdYn6A8RuUglqsJoIFidcXqsHF'

access_key =  '1368615358404395013-PxcHrwoqgibn6cJ0lDQD7WyCMuMwU5'

access_secret = 'sAK3yxL3y35C0oTrN92eb95YDzsUp7mGR1B9EO5bLrOFT'

API key: RnHS3Y3lkrv9pzkHVFcVYVArH \\
API secret key: xLMxTxhEHluPlcsPgb2rur07hBR4qSuzGdIVhJg0hpEiR1nTBm \\
Bearer token: 
AAAAAAAAAAAAAAAAAAAAAJ6bOAEAAAAAx7r6xai0HEcEhCREYGRBCePMops%3DnI0dVPhA0eFSBFrlc6M33YznpLELExxZGq0Mv3ZTXUTaj8C7Sq \\
Access token: 1376070608950267904-AlJkcNLLxsAByp9NmFU854synmuzq5 \\
Access token secret: dtH18I0v8Ob44BAUXhrJB9TNvnFfQykBafXY4fl4JtdJB

## Section 1

In [ ]:
import re
import numpy as np
import pandas as pd 
import io
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, roc_curve
import matplotlib.pyplot as plt

**Load datasets**

In [ ]:
from google.colab import files
data_to_load = files.upload()

In [ ]:
train_data = pd.read_csv(io.BytesIO(data_to_load['covid-tweets-train.csv']), encoding = 'latin1') 
print(train_data.head(5))

In [ ]:
data_to_load = files.upload()

In [ ]:
test_data = pd.read_csv(io.BytesIO(data_to_load['covid-tweets-test.csv']), encoding = 'latin1') 
print(test_data.head(5))

**Tokenization func**

In [ ]:
def tokenization(data, col):
  nltk.download('punkt')

  # Create a new column in our DF that contains token lists instead of raw text
  data['tokens'] = data[col].apply(nltk.word_tokenize)

**Remove URL func**

In [ ]:
def remove_url(data, col, n):
  lst = []
  for i in range(0,n):
    lst.append(re.sub(r'http\S+', '', data[col][i]))
  data['url_removed'] = lst

**Predict on test data and evaluation**

In [ ]:
def evaluate(X_test, y_test, model):
  y_preds = model.predict(X_test)

  print('Test accuracy with our model:',accuracy_score(y_test,y_preds))

  # # Plot the ROC curve 
  fpr, tpr, thresholds = roc_curve(y_test, y_preds, pos_label = 1)

  plt.plot(fpr,tpr)
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Receiver Operator Characteristic (ROC) Curve for RF')
  plt.show()


**(A)**


In [ ]:
nega = 0
neut = 0
posi = 0
num_train = len(train_data)
num_test = len(test_data)
for sentiment in train_data['Sentiment']:
  if sentiment == '0':
    nega += 1
  elif sentiment == '1':
    neut += 1
  elif sentiment == '2':
    posi += 1
print('the proportion of negative sentiment: ', nega/num_train)
print('the proportion of neutral sentiment: ', neut/num_train)
print('the proportion of positive sentiment: ', posi/num_train)

**(B)**

In [ ]:
tokenization(train_data, 'OriginalTweet')
tokenization(test_data, 'OriginalTweet')

**(C)**

In [ ]:
remove_url(train_data, 'OriginalTweet', num_train)
tokenization(train_data, 'url_removed')
train_data

In [ ]:
remove_url(test_data, 'OriginalTweet', num_test)
tokenization(test_data, 'url_removed')
test_data

**(D)**

## Section 2

**Build initial dataset**

In [ ]:
import tweepy as tw
# Use credentials to authorize access
auth = tw.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_key, access_secret)
api = tw.API(auth, wait_on_rate_limit=True)

date_since = "2021-03-25"
search_with_filter = " -filter:retweets"
tweets = tw.Cursor(api.search,
              q = search_with_filter,
              lang="en",
              since=date_since).items(5000)

# a = []
# b = []
# for tweet in tweets:
#   a.append(tweet.text)
#   b.append(tweet.user.followers_count)
# tweet_dataset = pd.DataFrame(a, columns = ['tweet_content'])
# tweet_dataset['followers'] = b


KeyboardInterrupt: ignored

In [ ]:
c = []
for ob in tweet_dataset["followers"]:
  if 0<ob<=1000:
    c.append(0) # low number of followers
  elif 1000< ob <= 10000:
    c.append(1) # medium number of followers
  else:
    c.append(2) # high number of followes
tweet_dataset['label'] = c
print(tweet_dataset.head(10))

                                       tweet_content  followers  label
0  @TheTravellerBot I can’t give you my strongest...          2      0
1                                  @pblescbrr_ okeey       1510      1
2           my friend said my whistling is cute. tf.         96      0
3  @huzaifasarki @Auto_follow_b Follow for follow...        596      0
4  Oops!\nDOJ Relies on Gorsuch’s ‘Bostock’ Opini...       7753      1
5  @fabbgrat It's the week before Black Widow and...       1244      1
6                  @MsKwashie The way you laugh dier        354      0
7                         @OperaFantasm *goes feral*        203      0
8  @CannalongaC @maddow I did notice it on my iPh...      26666      2
9  7 April 1997. The Chemical Brothers released D...      62992      2


In [ ]:
low_proportion = (tweet_dataset['label'].tolist().count(0))
print(low_proportion)
medium_proportion = (tweet_dataset['label'].tolist().count(1))
print(medium_proportion)
high_proportion = (tweet_dataset['label'].tolist().count(2))
print(high_proportion)
print(len(a))

638
231
61
930
